In [697]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Analizamos el contenido del CSV

In [698]:
casas = pd.read_csv('train.csv')

In [699]:
casas.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [700]:
casas.shape

(1460, 81)

In [701]:
casas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [702]:
nulos=casas.isnull().sum().sort_values(ascending=False)
nulos.head(20)       

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64

## Analizaremos las columnas con valores nulos

Analizaremos que columnas deberiamos borrar del data frame, observamos que hay 19 columnas con datos nulos, unas columnas mas que otras. 

In [703]:
casas.PoolQC.value_counts()

Gd    3
Fa    2
Ex    2
Name: PoolQC, dtype: int64

In [704]:
casas[["SalePrice","PoolQC","PoolArea"]][casas.PoolQC.isnull()].head()
#El que haya un valor nulo en PoolQC es debido a que no hay alberca, podemos llenar los nulos con "Nohay" y aplicar ANOVA 
#para ver si realmente hay diferencia significativa en el precio segun el tamaño de la alberca o la usencia de ella

,SalePrice,PoolQC,PoolArea
0,208500,NaN,0
1,181500,NaN,0
2,223500,NaN,0
3,140000,NaN,0
4,250000,NaN,0


In [705]:
casas_alberca=casas[["SalePrice","PoolQC","PoolArea"]][casas.PoolQC.notnull()]
model = ols('SalePrice ~ C(PoolQC)', data=casas_alberca).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Si solo aplicamos ANOVA con la tabla de los datos de las casas con alberca, podemos ver que no hay diferencia 
#significativa en el precio segun el tamaño de la alberca 

,sum_sq,df,F,PR(>F)
C(PoolQC),1.143135e+11,2.0,1.627469,0.303985
Residual,1.404801e+11,4.0,NaN,NaN


In [706]:
casas["PoolQC"].fillna("Nohay", inplace=True)
#Llenamos los nulos con el valor Nohay

In [707]:
model = ols('SalePrice ~ C(PoolQC)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Podemos observar que en este caso si hay diferencia significativa en el precio segun si la casa tiene o no alberca. 
#Para el análisis mas que quedarnos con la columna que se refiere al área de la alberca, crearemos una columna que nos indique si hay 
#o no alberca ya que si existía, no había diferencia significativa en los precios de acuerdo al tamaño. 

,sum_sq,df,F,PR(>F)
C(PoolQC),1.951702e+11,3.0,10.509853,7.700989e-07
Residual,9.012741e+12,1456.0,NaN,NaN


In [708]:
casas["Pool"]=np.where(casas.PoolArea!=0,1, casas.PoolArea)

In [709]:
casas.drop(columns=["PoolArea","PoolQC"], inplace=True)

Ahora analizaremos la siguiente columna con mayor cantidad de nulos 

In [710]:
casas.MiscFeature.value_counts()
#Se refiere a otras caracteristicas adicionales que tenga la casa, vamos a hacer algo parecido a lo que hicimos en la
#columna PollQC

Shed    49
Othr     2
Gar2     2
TenC     1
Name: MiscFeature, dtype: int64

In [711]:
casas_feat=casas[["SalePrice","MiscFeature"]][casas.MiscFeature.notnull()]
model = ols('SalePrice ~ C(MiscFeature)', data=casas_feat).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#No hay diferencia significativa en el precio segun las casas tengan alguna de estas amenidades extras 

,sum_sq,df,F,PR(>F)
C(MiscFeature),1.705983e+10,3.0,2.157324,0.104728
Residual,1.317978e+11,50.0,NaN,NaN


In [712]:
casas["MiscFeature"].fillna("Nohay", inplace=True)

In [713]:
model = ols('SalePrice ~ C(MiscFeature)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Podemos observar que no hay una diferencia significativa en los precios de las casas si estas tienen o no amenidades extras
#por lo que podemos no usar esta columna en el analisis. 

,sum_sq,df,F,PR(>F)
C(MiscFeature),6.518974e+10,4.0,2.593622,0.035004
Residual,9.142722e+12,1455.0,NaN,NaN


In [714]:
casas.drop(["MiscFeature"], axis=1, inplace=True)

In [715]:
casas.Alley.value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [716]:
casas_Alley=casas[["SalePrice","Alley"]][casas.Alley.notnull()]
model = ols('SalePrice ~ C(Alley)', data=casas_Alley).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Podemos observar que si existen diferencias significativas en el precio de las casas segun el tipo de acceso al callejon 

,sum_sq,df,F,PR(>F)
C(Alley),4.721637e+10,1.0,35.56206,4.899826e-08
Residual,1.181669e+11,89.0,NaN,NaN


In [717]:
casas["Alley"].fillna("Nohayacceso", inplace=True)

In [718]:
model = ols('SalePrice ~ C(Alley)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Podemos observar que sigue habiendo diferencias significativas en el precio segun el tipo de acceso al callejon o no,
#entonces mantendremos la info de esta columna y mas adelante la convertiremos a dummies.

,sum_sq,df,F,PR(>F)
C(Alley),1.879109e+11,2.0,15.176614,2.996380e-07
Residual,9.020000e+12,1457.0,NaN,NaN


In [719]:
casas.Fence.value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

In [720]:
casas_fence=casas[["SalePrice","Fence"]][casas.Fence.notnull()]
model = ols('SalePrice ~ C(Fence)', data=casas_fence).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Si hay diferencia significativa en el precio segun el tipo de barda que tenga la casa

,sum_sq,df,F,PR(>F)
C(Fence),5.494735e+10,3.0,4.948159,0.002313
Residual,1.025325e+12,277.0,NaN,NaN


In [721]:
casas["Fence"].fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(Fence)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Aquí podemos observar que si hay diferencia significativa en el precio segun el tipo de cerca que tenga, incluyendo
#si no tiene cerca así que mas adelante la convertiremos en dummies

,sum_sq,df,F,PR(>F)
C(Fence),3.279372e+11,4.0,13.433276,9.379977e-11
Residual,8.879974e+12,1455.0,NaN,NaN


In [722]:
casas.FireplaceQu.value_counts() 

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [723]:
casas_fire=casas[["SalePrice","FireplaceQu"]][casas.FireplaceQu.notnull()]
model = ols('SalePrice ~ C(FireplaceQu)', data=casas_fire).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Si hay diferencia significativa en el precio segun el tipo de chimenea que tenga la casa

,sum_sq,df,F,PR(>F)
C(FireplaceQu),6.561883e+11,4.0,24.398929,5.016300e-19
Residual,5.143505e+12,765.0,NaN,NaN


In [724]:
casas["FireplaceQu"].fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(FireplaceQu)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Podemos ver que si hay diferencia significativa en los precios segun la calidad de las chimenesas o si no hay 

,sum_sq,df,F,PR(>F)
C(FireplaceQu),2.706765e+12,5.0,121.075121,2.971217e-107
Residual,6.501147e+12,1454.0,NaN,NaN


In [725]:
#Como es una variable ordinal, la volveremos númerica con la siguiente asignación de valores
Fire1=np.where(casas.FireplaceQu=="Nohay",0, casas.FireplaceQu)
Fire2=np.where(Fire1=="Ex",5,Fire1)
Fire3=np.where(Fire2=="Gd",4,Fire2)
Fire4=np.where(Fire3=="TA",3,Fire3)
Fire5=np.where(Fire4=="Fa",2,Fire4)
casas["FireplaceQu"]=np.where(Fire5=="Po",1,Fire5)

In [726]:
casas.FireplaceQu=casas.FireplaceQu.astype(int)

In [727]:
casas.LotFrontage.value_counts()
#Aquí podemos ver que son valores numericos y representan los pies lineales de conexion con la calle por lo que inferimos que si hay un valor nulo es porque ese valor es cero,
#entonces rrellenaremos los valores nulos con ceros

60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
112.0      1
138.0      1
149.0      1
38.0       1
111.0      1
Name: LotFrontage, Length: 110, dtype: int64

In [728]:
casas.LotFrontage.fillna(0, inplace=True)
#Mas adelante analizaremos que tan correlacionada esta esta variable con el precio de las casas

Analizaremos otras variables, serán las de garage, que por lo que observo tienen el mismo número de nulos y es porque no hay garage, entonces lo que haremos, será rellenar los nulos y en el caso de las variables categoricas, observar si hay diferencia significativa con los precios, si es así las mantendremos, convirtiendolas a numéricas si son ordinales o mas adelante obteniendo sus dummies. En el caso de las columnas numericas, las analizaremos mas adelante.

In [729]:
casas.GarageType.value_counts()    

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

In [730]:
casas.GarageType.fillna("Nohay", inplace=True)

In [731]:
model = ols('SalePrice ~ C(GarageType)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencias significativas en el precio, segun esta variable, vamos a combertirla a dummies

,sum_sq,df,F,PR(>F)
C(GarageType),2.294650e+12,6.0,80.379992,6.117026e-87
Residual,6.913261e+12,1453.0,NaN,NaN


In [732]:
casas.GarageQual.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ GarageQual', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun la calidad del garage, dejaremos esta columna, 
#solo la convertiremos a numérica 

,sum_sq,df,F,PR(>F)
GarageQual,7.497217e+11,5.0,25.776093,5.388762e-25
Residual,8.458190e+12,1454.0,NaN,NaN


In [733]:
Gar1=np.where(casas.GarageQual=="Nohay",0, casas.GarageQual)
Gar2=np.where(Gar1=="Ex",5,Gar1)
Gar3=np.where(Gar2=="Gd",4,Gar2)
Gar4=np.where(Gar3=="TA",3,Gar3)
Gar5=np.where(Gar4=="Fa",2,Gar4)
casas["GarageQual"]=np.where(Gar5=="Po",1,Gar5)

In [734]:
casas.GarageQual=casas.GarageQual.astype(int)

In [893]:
#En el caso de esta columna,vamos a poner en el año de la construcción del garage el año de la
#construcción de la casa en el caso de que no haya garage solo rellenamos 81 datos
casas.GarageYrBlt.fillna("Nohay", inplace=True)
casas["GarageYrBlt"]=np.where(casas.GarageYrBlt=="Nohay",casas.YearBuilt, casas.GarageYrBlt)


In [737]:
casas.GarageYrBlt=casas.GarageYrBlt.astype(int)

In [738]:
casas.GarageCond.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(GarageCond)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun la calidad del garage, dejaremos esta columna, 
#solo la convertiremos a numérica 

,sum_sq,df,F,PR(>F)
C(GarageCond),7.490286e+11,5.0,25.750153,5.711746e-25
Residual,8.458883e+12,1454.0,NaN,NaN


In [739]:
Gar1=np.where(casas.GarageCond=="Nohay",0, casas.GarageCond)
Gar2=np.where(Gar1=="Ex",5,Gar1)
Gar3=np.where(Gar2=="Gd",4,Gar2)
Gar4=np.where(Gar3=="TA",3,Gar3)
Gar5=np.where(Gar4=="Fa",2,Gar4)
casas["GarageCond"]=np.where(Gar5=="Po",1,Gar5)

In [740]:
casas.GarageCond=casas.GarageCond.astype(int)

In [741]:
casas.GarageFinish.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(GarageFinish)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun si se termino el garage, dejaremos esta columna, 
#solo la convertiremos a numerica 

,sum_sq,df,F,PR(>F)
C(GarageFinish),2.816458e+12,3.0,213.867028,6.228747e-115
Residual,6.391453e+12,1456.0,NaN,NaN


In [742]:
Gar1=np.where(casas.GarageFinish=="Nohay",0, casas.GarageFinish)
Gar2=np.where(Gar1=="Fin",3,Gar1)
Gar3=np.where(Gar2=="RFn",2,Gar2)
casas["GarageFinish"]=np.where(Gar3=="Unf",1,Gar3)

In [743]:
casas.GarageFinish=casas.GarageFinish.astype(int)

Analizamos otras columnas referentes al sotano, aquí podemos ver claramente que si hay un valor nulo significa que no hay sotano, rellenaremos los nulos y analizaremos si dejamos o no la columna.

In [744]:
casas.BsmtQual.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(BsmtQual)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun la calidad del sotano, dejaremos esta columna, 
#solo la convertiremos a numerica 

,sum_sq,df,F,PR(>F)
C(BsmtQual),4.281621e+12,4.0,316.148635,8.158548e-196
Residual,4.926290e+12,1455.0,NaN,NaN


In [745]:
Bas1=np.where(casas.BsmtQual=="Nohay",0, casas.BsmtQual)
Bas2=np.where(Bas1=="Ex",5,Bas1)
Bas3=np.where(Bas2=="Gd",4,Bas2)
Bas4=np.where(Bas3=="TA",3,Bas3)
Bas5=np.where(Bas4=="Fa",2,Bas4)
casas["BsmtQual"]=np.where(Bas5=="Po",1,Bas5)

In [746]:
casas.BsmtQual=casas.BsmtQual.astype(int)

In [747]:
casas.BsmtCond.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(BsmtCond)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun la calidad del garage, dejaremos esta columna, 
#solo la convertiremos a numérica 

,sum_sq,df,F,PR(>F)
C(BsmtCond),4.732480e+11,4.0,19.708139,8.195794e-16
Residual,8.734663e+12,1455.0,NaN,NaN


In [748]:
Bas1=np.where(casas.BsmtCond=="Nohay",0, casas.BsmtCond)
Bas2=np.where(Bas1=="Ex",5,Bas1)
Bas3=np.where(Bas2=="Gd",4,Bas2)
Bas4=np.where(Bas3=="TA",3,Bas3)
Bas5=np.where(Bas4=="Fa",2,Bas4)
casas["BsmtCond"]=np.where(Bas5=="Po",1,Bas5)

In [749]:
casas.BsmtCond=casas.BsmtCond.astype(int)

In [750]:
casas.BsmtExposure.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(BsmtExposure)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencia significativa en los precios segun la calidad del garage, dejaremos esta columna, 
#solo la convertiremos a numérica 

,sum_sq,df,F,PR(>F)
C(BsmtExposure),1.376586e+12,4.0,63.939761,7.557758e-50
Residual,7.831326e+12,1455.0,NaN,NaN


In [751]:
Bas1=np.where(casas.BsmtExposure=="Nohay",0, casas.BsmtExposure)
Bas2=np.where(Bas1=="Gd",4,Bas1)
Bas3=np.where(Bas2=="Av",3,Bas2)
Bas4=np.where(Bas3=="Mn",2,Bas3)
casas["BsmtExposure"]=np.where(Bas4=="No",1,Bas4)

In [752]:
casas.BsmtExposure=casas.BsmtExposure.astype(int)

In [753]:
casas.BsmtFinType1.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(BsmtFinType1)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencias significativas en el precio, segun esta variable, vamos a combertirla a dummies

,sum_sq,df,F,PR(>F)
C(BsmtFinType1),1.941124e+12,6.0,64.6882,2.386358e-71
Residual,7.266788e+12,1453.0,NaN,NaN


In [754]:
casas.BsmtFinType2.fillna("Nohay", inplace=True)
model = ols('SalePrice ~ C(BsmtFinType2)', data=casas).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Como si hay diferencias significativas en el precio, segun esta variable, vamos a combertirla a dummies

,sum_sq,df,F,PR(>F)
C(BsmtFinType2),2.789443e+11,6.0,7.565378,5.225649e-08
Residual,8.928967e+12,1453.0,NaN,NaN


Checaremos mas columnas con nulos. Analizando las columnas MasVnrArea y MasVnrType podemos observar que el hecho de que el área sea cero, significa que no tiene ningun tipo de mampostería  en su mayoría, por lo que los nulos de ambas columnas, no significa la usencia de mampostería, sino realmente que no sabemos de que tipo y que área ocupan. 

In [755]:
casas.MasVnrType.value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [756]:
casas[casas.MasVnrArea==0].MasVnrType.value_counts()
#Aquí podemos ver que la mayoria de los datos donde el área es cero el tipo de mampostería es cero 

None       859
BrkFace      1
Stone        1
Name: MasVnrType, dtype: int64

In [757]:
casas[casas.MasVnrArea.isnull()].MasVnrType
#Los valores nulos coinciden para ambas columnas, analizaremos que tan importante es el tipo de mampostería para el 
#precio de la casa y si lo es, lo mejor sería deshacernos de esas pocas filas en que hay nulos 

234     NaN
529     NaN
650     NaN
936     NaN
973     NaN
977     NaN
1243    NaN
1278    NaN
Name: MasVnrType, dtype: object

In [758]:
casas_ma=casas[["SalePrice","MasVnrType"]][casas.MasVnrType.notnull()]
model = ols('SalePrice ~ C(MasVnrType)', data=casas_ma).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table
#Si hay diferencias significativas entre los precios y los datos de esta columna entonces si la mantendremos pero 
#borraremos las filas con los valores nulos 

,sum_sq,df,F,PR(>F)
C(MasVnrType),1.713827e+12,3.0,111.67238,4.793331e-65
Residual,7.407445e+12,1448.0,NaN,NaN


Lo mismo pasa con el valor nulo en la columna de Electrical, no significa que no tenga sistema electrico, solo no sabemos de que tipo es, por lo que lo eliminaremos. 

In [759]:
casas[casas.Electrical.isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,3SsnPorch,ScreenPorch,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Pool
1379,1380,80,RL,73.0,9735,Pave,Nohayacceso,Reg,Lvl,AllPub,...,0,0,Nohay,0,5,2008,WD,Normal,167500,0


In [760]:
casas1=casas[casas.Electrical.notnull()]
Casas=casas1[casas1.MasVnrType.notnull()]

In [761]:
Casas.shape

(1451, 79)

## Análisis de las columnas no numéricas con ANOVA

In [762]:
Casas_num=Casas._get_numeric_data()
Casas_num
#Observamos que hay 45 columnas numericas de 79 es decir que hay 34 columnas de valores no numéricos, 
#veremos cuales podemos poner como ordnales

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtQual,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,MoSold,YrSold,SalePrice,Pool
0,1,60,65.0,8450,7,5,2003,2003,196.0,4,...,0,61,0,0,0,0,2,2008,208500,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,4,...,298,0,0,0,0,0,5,2007,181500,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,4,...,0,42,0,0,0,0,9,2008,223500,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,3,...,0,35,272,0,0,0,2,2006,140000,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,4,...,192,84,0,0,0,0,12,2008,250000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,4,...,0,40,0,0,0,0,8,2007,175000,0
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,4,...,349,0,0,0,0,0,2,2010,210000,0
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,3,...,0,60,0,0,0,2500,5,2010,266500,0
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,3,...,366,0,112,0,0,0,4,2010,142125,0


In [763]:
col_tol=list(Casas.columns)
col_tol=set(col_tol)
col_num=list(Casas_num.columns)
col_num=set(col_num)
col_no_num=list(col_tol-col_num)
col_no_num

['Heating',
 'ExterCond',
 'Exterior1st',
 'LotConfig',
 'SaleType',
 'KitchenQual',
 'Electrical',
 'LandContour',
 'BsmtFinType1',
 'SaleCondition',
 'RoofMatl',
 'Functional',
 'LotShape',
 'CentralAir',
 'Condition1',
 'Street',
 'BsmtFinType2',
 'MasVnrType',
 'MSZoning',
 'HeatingQC',
 'Condition2',
 'GarageType',
 'Fence',
 'ExterQual',
 'RoofStyle',
 'Neighborhood',
 'PavedDrive',
 'BldgType',
 'Utilities',
 'Foundation',
 'Alley',
 'Exterior2nd',
 'HouseStyle',
 'LandSlope']

In [764]:
pval=[]
for i in col_no_num:
    model = ols(f'SalePrice ~ C({i})', data=Casas).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    pval.append((i,anova_table["PR(>F)"][0]))


In [765]:
sorted(pval, key=lambda x:x[1], reverse=True)
#Con esto podemos darnos cuenta de que cuales columnas omitir en el analisis, si el p value es mayor que 0.05 entonces no hay 
#diferencia significativa de los precios, segun los diferentes valores de la columna. Entonces podemos omitir Utilities
#LandSlope y Street

[('Utilities', 0.5866752889962392),
 ('LandSlope', 0.13127011944509862),
 ('Street', 0.11859948981409002),
 ('Condition2', 0.04321456724041051),
 ('Heating', 0.0007803659466487865),
 ('Functional', 0.0004700459415538208),
 ('LotConfig', 2.290629428018329e-06),
 ('ExterCond', 5.896037266145833e-07),
 ('Alley', 3.181711527904088e-07),
 ('Condition1', 1.0254754788825274e-07),
 ('RoofMatl', 6.2644707707918e-08),
 ('BsmtFinType2', 6.155118310899127e-08),
 ('LandContour', 2.4428259821611356e-08),
 ('BldgType', 2.5621049717974e-10),
 ('Fence', 1.3142431210924415e-10),
 ('RoofStyle', 1.5751265482985026e-17),
 ('PavedDrive', 2.2756954218098116e-18),
 ('Electrical', 2.0828683148989303e-18),
 ('CentralAir', 2.201597653808998e-22),
 ('HouseStyle', 2.9096706393409073e-25),
 ('LotShape', 1.583525201820787e-25),
 ('MSZoning', 1.1766305337800056e-34),
 ('SaleType', 4.225136766160507e-41),
 ('Exterior2nd', 1.1334611089486893e-42),
 ('SaleCondition', 6.8673124919720235e-43),
 ('Exterior1st', 5.648843694

In [ ]:
Casas.drop(columns=["Utilities",'LandSlope','Street',"Id"],axis=1, inplace=True)

## Simplificación de columnas 

Vamos a ver si podemos simplificar algunas columnas en una sola 

In [767]:
SalePrice=Casas.SalePrice

In [768]:
Casas.drop(columns=["SalePrice"], axis=1, inplace=True)

In [769]:
Casas.insert(0, 'SalePrice', SalePrice)

In [770]:
corr=Casas._get_numeric_data().corr()
corr["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790085
GrLivArea        0.710073
GarageCars       0.639749
GarageArea       0.622484
TotalBsmtSF      0.613299
1stFlrSF         0.606969
BsmtQual         0.584317
FullBath         0.562710
GarageFinish     0.549512
TotRmsAbvGrd     0.536366
YearBuilt        0.523274
FireplaceQu      0.520946
GarageYrBlt      0.508230
YearRemodAdd     0.507488
MasVnrArea       0.477486
Fireplaces       0.468973
BsmtFinSF1       0.383995
BsmtExposure     0.372754
WoodDeckSF       0.324658
2ndFlrSF         0.322840
OpenPorchSF      0.311244
HalfBath         0.282331
GarageQual       0.274211
LotArea          0.264668
GarageCond       0.263500
BsmtFullBath     0.224988
BsmtUnfSF        0.215707
BsmtCond         0.208984
LotFrontage      0.208453
BedroomAbvGr     0.171956
ScreenPorch      0.113017
Pool             0.094415
3SsnPorch        0.045235
MoSold           0.045085
BsmtFinSF2      -0.010350
BsmtHalfBath    -0.016021
MiscVal         -0.020961
LowQualFinSF

Nos damos cuenta de que hay variables que pueden simplificarse en una sola como el número de baños y medios baños, en los pisos de arriba y en el sotano si checamos sus correlaciones con el precio de la casa, la correlación mas alta es el baño completo, por lo que podemos juntar estas columnas en una sola que sea el número de baños en total 

In [771]:
Casas["TotalBath"]=(Casas["HalfBath"]*(0.5)+Casas["FullBath"])+(Casas["BsmtHalfBath"]*0.5+Casas["BsmtFullBath"])

<ipython-input-771-94522eda7c3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["TotalBath"]=(Casas["HalfBath"]*(0.5)+Casas["FullBath"])+(Casas["BsmtHalfBath"]*0.5+Casas["BsmtFullBath"])


In [772]:
corr=Casas._get_numeric_data().corr()
corr["SalePrice"]["TotalBath"]
#Podemos observar que hay una mayor correlación con esta columna que con las columnas de baños por separado 

0.632059536288091

In [773]:
Casas.drop(columns=["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"], axis=1, inplace=True)

In [774]:
#Observamos que las columnas GarageCars y  representan lo mismo, sinembargo Garage cars
#tiene una mayor correlación con el precio, por lo que dejaremos solamente esta columna
Casas.drop(columns=["GarageArea"], axis=1, inplace=True)

## Convertimos las columnas ordinales que faltan en numéricas 

In [775]:
Casas.Functional.value_counts()

Typ     1352
Min2      34
Min1      31
Mod       15
Maj1      13
Maj2       5
Sev        1
Name: Functional, dtype: int64

In [776]:
B1=np.where(Casas.Functional=="Typ",8,Casas.Functional)
B2=np.where(B1=="Min1",7,B1)
B3=np.where(B2=="Min2",6,B2)
B4=np.where(B3=="Mod",5,B3)
B5=np.where(B4=="Maj1",4,B4)
B6=np.where(B5=="Maj2",3,B5)
B7=np.where(B6=="Sev",2,B6)
Casas["Functional"]=np.where(B7=="Sal",1,B7)

<ipython-input-776-bab4f680dde2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["Functional"]=np.where(B7=="Sal",1,B7)


In [777]:
Casas.Functional=Casas.Functional.astype(int)

C:\Users\Tania\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [778]:
Casas.ExterCond.value_counts()

TA    1273
Gd     146
Fa      28
Ex       3
Po       1
Name: ExterCond, dtype: int64

In [779]:
B1=np.where(Casas.ExterCond=="Ex",5,Casas.ExterCond)
B2=np.where(B1=="Gd",4,B1)
B3=np.where(B2=="TA",3,B2)
B4=np.where(B3=="Fa",2,B3)
Casas["ExterCond"]=np.where(B4=="Po",1,B4)

<ipython-input-779-b4ba24d6aa6f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["ExterCond"]=np.where(B4=="Po",1,B4)


In [780]:
Casas.ExterCond=Casas.ExterCond.astype(int)

In [781]:
Casas.LandContour.value_counts()

Lvl    1302
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

In [782]:
B1=np.where(Casas.LandContour=="Lvl",1,Casas.LandContour)
B2=np.where(B1=="Bnk",2,B1)
B3=np.where(B2=="HLS",3,B2)
Casas["LandContour"]=np.where(B3=="Low",4,B3)

<ipython-input-782-8b9b1a3d1053>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["LandContour"]=np.where(B3=="Low",4,B3)


In [783]:
Casas.LandContour=Casas.LandContour.astype(int)

In [784]:
Casas.PavedDrive.value_counts()

Y    1331
N      90
P      30
Name: PavedDrive, dtype: int64

In [785]:
B1=np.where(Casas.PavedDrive=="N",0,Casas.PavedDrive)
B2=np.where(B1=="Y",1,B1)
Casas["PavedDrive"]=np.where(B2=="P",0.5,B2)

<ipython-input-785-1d8881beae63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["PavedDrive"]=np.where(B2=="P",0.5,B2)


In [786]:
Casas.PavedDrive=Casas.PavedDrive.astype(int)

In [787]:
Casas.CentralAir.value_counts()

Y    1356
N      95
Name: CentralAir, dtype: int64

In [788]:
B1=np.where(Casas.CentralAir=="N",0,Casas.CentralAir)
Casas["CentralAir"]=np.where(B1=="Y",1,B1)

<ipython-input-788-27530349a2a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["CentralAir"]=np.where(B1=="Y",1,B1)


In [789]:
Casas.CentralAir=Casas.CentralAir.astype(int)

In [790]:
Casas.LotShape.value_counts()

Reg    918
IR1    482
IR2     41
IR3     10
Name: LotShape, dtype: int64

In [791]:
B1=np.where(Casas.LotShape=="Reg",4,Casas.LotShape)
B2=np.where(B1=="IR1",3,B1)
B3=np.where(B2=="IR2",2,B2)
Casas["LotShape"]=np.where(B3=="IR3",1,B3)

<ipython-input-791-b60941d2d9b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["LotShape"]=np.where(B3=="IR3",1,B3)


In [792]:
Casas.LotShape=Casas.LotShape.astype(int)

In [793]:
Casas.HeatingQC.value_counts()

Ex    734
TA    427
Gd    240
Fa     49
Po      1
Name: HeatingQC, dtype: int64

In [794]:
B1=np.where(Casas.HeatingQC=="Ex",5,Casas.HeatingQC)
B2=np.where(B1=="Gd",4,B1)
B3=np.where(B2=="TA",3,B2)
B4=np.where(B3=="Fa",2,B3)
Casas["HeatingQC"]=np.where(B4=="Po",1,B4)

<ipython-input-794-c6cc8561bd2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["HeatingQC"]=np.where(B4=="Po",1,B4)


In [795]:
Casas.HeatingQC=Casas.HeatingQC.astype(int)

In [796]:
Casas.KitchenQual.value_counts()

TA    734
Gd    579
Ex     99
Fa     39
Name: KitchenQual, dtype: int64

In [797]:
B1=np.where(Casas.KitchenQual=="Ex",5,Casas.KitchenQual)
B2=np.where(B1=="Gd",4,B1)
B3=np.where(B2=="TA",3,B2)
B4=np.where(B3=="Fa",2,B3)
Casas["KitchenQual"]=np.where(B4=="Po",1,B4)

<ipython-input-797-535ca85e8e79>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["KitchenQual"]=np.where(B4=="Po",1,B4)


In [798]:
Casas.KitchenQual=Casas.KitchenQual.astype(int)

In [799]:
Casas.ExterQual.value_counts()

TA    905
Gd    481
Ex     51
Fa     14
Name: ExterQual, dtype: int64

In [800]:
B1=np.where(Casas.ExterQual=="Ex",5,Casas.ExterQual)
B2=np.where(B1=="Gd",4,B1)
B3=np.where(B2=="TA",3,B2)
B4=np.where(B3=="Fa",2,B3)
Casas["ExterQual"]=np.where(B4=="Po",1,B4)

<ipython-input-800-019658e9932f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Casas["ExterQual"]=np.where(B4=="Po",1,B4)


In [801]:
Casas.ExterQual=Casas.ExterQual.astype(int)

## Analizamos los coeficientes de correlación de todas las columnas númericas incluidas las ordinales 

In [811]:
numericas=Casas._get_numeric_data()
corr=numericas.corr()
corr["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790085
GrLivArea        0.710073
ExterQual        0.681200
KitchenQual      0.658647
GarageCars       0.639749
TotalBath        0.632060
TotalBsmtSF      0.613299
1stFlrSF         0.606969
BsmtQual         0.584317
GarageFinish     0.549512
TotRmsAbvGrd     0.536366
YearBuilt        0.523274
FireplaceQu      0.520946
GarageYrBlt      0.508230
YearRemodAdd     0.507488
MasVnrArea       0.477486
Fireplaces       0.468973
HeatingQC        0.427602
BsmtFinSF1       0.383995
BsmtExposure     0.372754
WoodDeckSF       0.324658
2ndFlrSF         0.322840
OpenPorchSF      0.311244
GarageQual       0.274211
LotArea          0.264668
GarageCond       0.263500
CentralAir       0.251584
PavedDrive       0.232009
BsmtUnfSF        0.215707
BsmtCond         0.208984
LotFrontage      0.208453
BedroomAbvGr     0.171956
ScreenPorch      0.113017
Functional       0.109545
Pool             0.094415
LandContour      0.074561
3SsnPorch        0.045235
MoSold      

In [813]:
corr=numericas.corr()
corr1=corr[["SalePrice"]]
corr2=corr1[(corr1.SalePrice>=0.5) | (corr1.SalePrice<=-0.5)]
corr2

,SalePrice
SalePrice,1.000000
OverallQual,0.790085
YearBuilt,0.523274
YearRemodAdd,0.507488
ExterQual,0.681200
BsmtQual,0.584317
TotalBsmtSF,0.613299
1stFlrSF,0.606969
GrLivArea,0.710073
KitchenQual,0.658647


## Convertimos las columnas de valores categoricos en dummies y analizamos su correlación

In [809]:
cat=list(set(Casas.columns)-set(Casas._get_numeric_data()))
categoricas=Casas[cat]

In [814]:
categoricas_dum=pd.get_dummies(categoricas)

In [827]:
total=pd.concat([numericas,categoricas_dum], axis=1)
total.shape

(1451, 224)

In [860]:
corr=total.corr()
corr1=corr[["SalePrice"]]
corr2=corr1[(corr1.SalePrice>=0.5) | (corr1.SalePrice<=-0.5)]
corr2
#Vemos que no hubo otra columna dentro de las dummies que tenga una correlación de 0.5 o mas. 
#Estas son las que usaremos para nuestro modelo

,SalePrice
SalePrice,1.000000
OverallQual,0.790085
YearBuilt,0.523274
YearRemodAdd,0.507488
ExterQual,0.681200
BsmtQual,0.584317
TotalBsmtSF,0.613299
1stFlrSF,0.606969
GrLivArea,0.710073
KitchenQual,0.658647


## Aplicamos PCA para las columnas con coef. de correlación entre 0 y 0.5 o entre -0.5 y 0

In [861]:
todas=set(corr1.index)
especiales=set(corr2.index)
features=list(todas-especiales)
len(features)

208

Aplicaremos PCA para las otras variables 

In [831]:
X = total[features]
y = total['SalePrice']
scaler = StandardScaler()

In [832]:
stand_data = scaler.fit_transform(X)
df_stand_data = pd.DataFrame(stand_data, index = X.index, columns = X.columns)

In [833]:
df_stand_data.head()

,ExterCond,SaleType_CWD,SaleCondition_Alloca,HouseStyle_1Story,LandContour,RoofMatl_Roll,Foundation_PConc,Functional,BsmtFinType1_BLQ,GarageType_Nohay,...,KitchenAbvGr,Foundation_Wood,Exterior1st_CemntBd,Electrical_SBrkr,Exterior1st_WdShing,Heating_GasW,BsmtFinType1_Rec,MasVnrType_BrkFace,BldgType_Twnhs,Exterior2nd_ImStucc
0,-0.238892,-0.052577,-0.087401,-0.993816,-0.307183,-0.026261,1.128847,0.236395,-0.337022,-0.243154,...,-0.21037,-0.045517,-0.205876,0.307032,-0.135076,-0.112076,-0.316348,1.503554,-0.174756,-0.083304
1,-0.238892,-0.052577,-0.087401,1.006222,-0.307183,-0.026261,-0.885860,0.236395,-0.337022,-0.243154,...,-0.21037,-0.045517,-0.205876,0.307032,-0.135076,-0.112076,-0.316348,-0.665091,-0.174756,-0.083304
2,-0.238892,-0.052577,-0.087401,-0.993816,-0.307183,-0.026261,1.128847,0.236395,-0.337022,-0.243154,...,-0.21037,-0.045517,-0.205876,0.307032,-0.135076,-0.112076,-0.316348,1.503554,-0.174756,-0.083304
3,-0.238892,-0.052577,-0.087401,-0.993816,-0.307183,-0.026261,-0.885860,0.236395,-0.337022,-0.243154,...,-0.21037,-0.045517,-0.205876,0.307032,-0.135076,-0.112076,-0.316348,-0.665091,-0.174756,-0.083304
4,-0.238892,-0.052577,-0.087401,-0.993816,-0.307183,-0.026261,1.128847,0.236395,-0.337022,-0.243154,...,-0.21037,-0.045517,-0.205876,0.307032,-0.135076,-0.112076,-0.316348,1.503554,-0.174756,-0.083304


In [834]:
pca = PCA()
pca.fit(df_stand_data)
pca.explained_variance_ratio_

array([4.57213729e-02, 3.30472533e-02, 2.38116263e-02, 2.17341915e-02,
       1.90222584e-02, 1.85242196e-02, 1.61780958e-02, 1.58855337e-02,
       1.46403661e-02, 1.37089194e-02, 1.33172439e-02, 1.23683623e-02,
       1.22473890e-02, 1.17757093e-02, 1.12529294e-02, 1.11721722e-02,
       1.10804288e-02, 1.07633272e-02, 1.06997630e-02, 1.05816929e-02,
       1.03685312e-02, 9.98604354e-03, 9.84778110e-03, 9.55808722e-03,
       9.31553887e-03, 9.15092573e-03, 9.08701795e-03, 8.88385788e-03,
       8.79088275e-03, 8.63523709e-03, 8.61279351e-03, 8.42687258e-03,
       8.33768568e-03, 8.10037678e-03, 7.94206501e-03, 7.86357115e-03,
       7.64230703e-03, 7.44916043e-03, 7.34678408e-03, 7.30760893e-03,
       7.13470392e-03, 7.10347974e-03, 7.05045129e-03, 6.87197940e-03,
       6.85564938e-03, 6.72676015e-03, 6.70303042e-03, 6.62396670e-03,
       6.56429591e-03, 6.50727824e-03, 6.36633960e-03, 6.31210972e-03,
       6.26981742e-03, 6.19845264e-03, 6.13608338e-03, 6.09185043e-03,
      

In [847]:
num=list(pca.explained_variance_ratio_.cumsum())
for i in range(0,len(num)):
    if 0.81>num[i]>=0.8:
        print(i)
        
#92 es el número de componentes principales a usar        

92
93


In [853]:
pca_bueno = PCA(n_components=92)
pca_bueno1 = pca_bueno.fit_transform(df_stand_data)

In [854]:
colu1=[f"C{i}" for i in range(1,93)]


In [855]:
df_pca = pd.DataFrame(pca_bueno1, index=df_stand_data.index,columns = colu1)

In [856]:
df_pca
#Son los valores de los componentes principales 

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92
0,-2.942131,-1.294634,0.251941,0.284031,0.693073,-2.431391,0.921978,0.601779,-1.032373,-0.383796,...,-0.007395,-0.242258,0.058476,0.735642,-0.233059,-0.514446,-0.399157,-0.343261,-0.072489,0.095544
1,-0.125681,2.664209,-1.056692,0.484955,-0.835900,0.403971,0.251281,0.845329,1.005054,1.886104,...,0.665471,2.384944,-0.523344,-2.591357,-0.217520,-1.477593,-0.608948,0.235741,1.955385,-1.377056
2,-3.366054,-1.208631,0.043861,-0.217690,1.156117,-2.095943,0.982240,0.659766,-1.023473,-0.354671,...,-0.095615,-0.171633,0.011905,-0.025889,-0.188333,-0.538634,-0.074701,-0.154408,-0.151136,0.202272
3,1.807893,-0.846435,-2.660686,-1.080505,1.125068,0.109565,-0.199937,0.101891,0.551020,-0.258932,...,0.429552,-0.631692,-1.014090,1.791428,0.118256,-0.377078,0.524930,-0.389426,1.241760,-1.453360
4,-4.189123,-0.916705,-0.278967,-0.547044,3.169354,-2.166406,0.877774,-0.635907,-1.571778,-0.345493,...,0.290413,0.497394,-0.527886,-1.074318,-0.451060,0.288607,-0.617805,1.497851,0.580827,0.247614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-1.812386,-2.258454,-0.504273,-0.987628,0.838705,-2.694660,1.239227,1.460940,0.966446,-0.521931,...,0.481853,0.211195,0.428360,-0.364168,-0.305806,0.987220,0.489792,0.156436,-0.164565,0.440150
1456,-0.450997,5.073652,0.019020,0.306081,-0.158666,0.839164,-0.281542,-0.187983,1.266468,0.667746,...,0.472571,0.515327,-0.067244,0.172672,0.463361,0.295207,0.360655,0.662009,0.090797,1.188538
1457,-0.435623,-0.529510,-2.886772,1.818603,2.829297,0.582948,-0.241939,-0.300664,-1.346295,3.410591,...,2.387708,-1.945260,0.386028,-1.618434,1.990850,-0.081518,-2.055533,1.257849,-1.569240,1.369057
1458,1.526966,3.676906,0.206272,-0.160255,-2.620317,1.099052,-2.092531,1.618040,-0.713067,0.939176,...,-0.006521,-0.555304,0.569490,-0.942331,-0.405385,0.596338,-0.431670,-1.076140,-1.069171,0.852032


In [857]:
df_pca["SalePrice"]=y

In [859]:
corr=df_pca.corr()
corr1=corr[["SalePrice"]]
corr2=corr1[(corr1.SalePrice>=0.5) | (corr1.SalePrice<=-0.5)]
corr2
#podemos observar que el único componente principal con mayor correlación con el precio de las casas es C1, por lo que 
#será la unica columna extra que usaremos para nuestro modelo

,SalePrice
C1,-0.676578
SalePrice,1.000000


## Regresión Lineal

In [865]:
X=total[especiales]
X.drop("SalePrice", axis=1, inplace=True)

In [867]:
X["C1"]=df_pca.C1

<ipython-input-867-1d820fe7cb72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["C1"]=df_pca.C1


In [868]:
X

,YearRemodAdd,OverallQual,KitchenQual,TotRmsAbvGrd,1stFlrSF,TotalBath,TotalBsmtSF,ExterQual,GarageYrBlt,GrLivArea,FireplaceQu,GarageFinish,YearBuilt,GarageCars,BsmtQual,C1
0,2003,7,4,8,856,3.5,856,4,2003,1710,0,2,2003,2,4,-2.942131
1,1976,6,3,6,1262,2.5,1262,3,1976,1262,3,2,1976,2,4,-0.125681
2,2002,7,4,6,920,3.5,920,4,2001,1786,3,2,2001,2,4,-3.366054
3,1970,7,4,7,961,2.0,756,3,1998,1717,4,1,1915,3,3,1.807893
4,2000,8,4,9,1145,3.5,1145,4,2000,2198,3,2,2000,3,4,-4.189123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2000,6,3,7,953,2.5,953,3,1999,1647,3,2,1999,2,4,-1.812386
1456,1988,6,3,7,2073,3.0,1542,3,1978,2073,3,1,1978,2,4,-0.450997
1457,2006,7,4,9,1188,2.0,1152,5,1941,2340,4,2,1941,1,3,-0.435623
1458,1996,5,4,5,1078,2.0,1078,3,1950,1078,0,1,1950,1,3,1.526966


In [887]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 22, train_size=0.80)

In [888]:
linreg = LinearRegression()

In [889]:
linreg.fit(X_train, y_train)

LinearRegression()

In [890]:
linreg.score(X_train, y_train)
#Aquí podemos ver que es alto el r cuadrado cuando entrenamos el modelo

0.8089644478053497

In [891]:
linreg.score(X_test, y_test)
#Baja un poco el r cuadrado con nuestra prueba 

0.7618737966866229

In [875]:
#Son los coeficientes del modelo con las columnas utilicades en el df X
linreg.coef_

array([-2.56204380e+01,  1.16215457e+04,  1.15291694e+04, -1.25088918e+03,
        2.67895565e+01,  8.60222948e+03,  4.15758851e-01,  1.62222108e+04,
       -2.17269715e+02,  4.21319321e+01,  1.92626601e+03, -5.45921571e+02,
       -7.66665196e+01,  1.06153921e+04,  5.05918472e+03, -4.85465976e+03])

In [876]:
#Es el término independiente del modelo
linreg.intercept_

500889.7094415222